In [1]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import Aer  
from qiskit.circuit.library.standard_gates import CXGate
from qiskit.circuit.library import XGate, CCXGate
from qiskit.quantum_info import Statevector
from ComparisonMaker import get_less_than_operations, get_less_than_logic_explanation
from CustomOperations import AND, OR, invAND, invOR, X

In [2]:
def lessthanlogic0(position_register, ancilla_register, comparison_register):
    logic = []
    logic.extend(OR(position_register[1], position_register[2], ancilla_register[1]))
    logic.extend(OR(position_register[3], ancilla_register[1], ancilla_register[2]))
    logic.extend(AND(position_register[4], ancilla_register[2], ancilla_register[3]))
    logic.extend(invOR(position_register[3], ancilla_register[1], ancilla_register[2]))
    logic.extend(invOR(position_register[1], position_register[2], ancilla_register[1]))

    logic.extend(AND(position_register[5], position_register[6], ancilla_register[1]))
    logic.extend(AND(ancilla_register[1], ancilla_register[3], ancilla_register[2]))
    logic.extend(OR(position_register[7], ancilla_register[2], comparison_register[0]))
    logic.extend(invAND(ancilla_register[1], ancilla_register[3], ancilla_register[2]))
    logic.extend(invAND(position_register[5], position_register[6], ancilla_register[1]))

    logic.extend(OR(position_register[1], position_register[2], ancilla_register[1]))
    logic.extend(OR(position_register[3], ancilla_register[1], ancilla_register[2]))
    logic.extend(invAND(position_register[4], ancilla_register[2], ancilla_register[3]))
    logic.extend(invOR(position_register[3], ancilla_register[1], ancilla_register[2]))
    logic.extend(invOR(position_register[1], position_register[2], ancilla_register[1]))
    return logic

def lessthanlogic1(position_register, ancilla_register, comparison_register):
    logic = []
    logic.extend(OR(position_register[0], position_register[1], ancilla_register[1]))
    logic.extend(OR(position_register[2], ancilla_register[1], ancilla_register[2]))
    logic.extend(OR(position_register[3], ancilla_register[2], ancilla_register[3]))
    logic.extend(invOR(position_register[2], ancilla_register[1], ancilla_register[2]))
    logic.extend(invOR(position_register[0], position_register[1], ancilla_register[1]))
    
    logic.extend(AND(position_register[4], position_register[5], ancilla_register[1]))
    logic.extend(AND(ancilla_register[1], ancilla_register[3], ancilla_register[2]))
    logic.extend(invAND(position_register[4], position_register[5], ancilla_register[1]))

    logic.extend(AND(position_register[6], ancilla_register[2], ancilla_register[1]))
    logic.extend(OR(position_register[7], ancilla_register[1], comparison_register[1]))
    return logic


In [3]:
QUBITS_NUM = 8
ANCILLARY_QUBITS_NUM = 4
COMPARATOR_QUBITS_NUM = 2
numbers = [68, 96]


x_0 = 1.5
p_0 = 1.0
δ = 1/3
N = 2**QUBITS_NUM
total_qubits = QUBITS_NUM + ANCILLARY_QUBITS_NUM + COMPARATOR_QUBITS_NUM
d = 10
Δ_x = (2*d) / N

In [4]:
def logics(position_register, ancilla_register, comparison_register):
    logic0 = lessthanlogic0(position_register, ancilla_register, comparison_register)
    logic1 = lessthanlogic1(position_register, ancilla_register, comparison_register)
    all_bps = {numbers[0]: logic0, numbers[1]: logic1} 
    return all_bps

def create_breakpoint_operations(number, logics, position_register, ancilla_register, comparison_register):
    return logics.get(number, [])

def create_combined_operations(numbers, n_bits, logics, position_register, ancilla_register, comparison_register):
    all_operations = []
    explanations = {}

    first_number = numbers[0]
    result1 = get_less_than_operations(first_number, n_bits)
    explanations[first_number] = result1["explanation"]

    x_gate_operations1 = []
    for bit_pos in result1["x_gate_positions"]:
        x_gate_operations1.append((XGate(), [position_register[bit_pos]]))
    
    bp_operations1 = create_breakpoint_operations(first_number, logics, position_register, ancilla_register, comparison_register)
    
    all_operations.extend(x_gate_operations1)
    all_operations.extend(bp_operations1)

    for bit_pos in result1["x_gate_positions"]:
        all_operations.append((XGate(), [position_register[bit_pos]]))

    second_number = numbers[1]
    result2 = get_less_than_operations(second_number, n_bits)
    explanations[second_number] = result2["explanation"]
    
    x_gate_operations2 = []
    for bit_pos in result2["x_gate_positions"]:
        x_gate_operations2.append((XGate(), [position_register[bit_pos]]))
    
    # Logic operations for second number
    bp_operations2 = create_breakpoint_operations(second_number, logics, position_register, ancilla_register, comparison_register)
    
    all_operations.extend(x_gate_operations2)
    all_operations.extend(bp_operations2)
    
    return all_operations, explanations

def check_all_breakpoints(main_circuit, operations):
    for gate, qubits in operations:
        main_circuit.append(gate, qubits)

def uncheck_all_breakpoints(main_circuit, operations):
    for gate, qubits in reversed(operations):
        main_circuit.append(gate, qubits)

def apply_x_gates(main_circuit, bit_positions, position_register):
    for bit_pos in bit_positions:
        main_circuit.append(XGate(), [position_register[bit_pos]])

def apply_comparison_logic(main_circuit, number, logic):
    bp_operations = logic.get(number, [])
    for gate, qubits in bp_operations:
        main_circuit.append(gate, qubits)

def handle_comparison(main_circuit, number, position_register, logic, undo_x_gates=False):
    result = get_less_than_operations(number, QUBITS_NUM)
    x_positions = result["x_gate_positions"]
    apply_x_gates(main_circuit, x_positions, position_register)
    apply_comparison_logic(main_circuit, number, logic)
    if undo_x_gates:
        apply_x_gates(main_circuit, x_positions, position_register)
    
    return result["explanation"], x_positions

def comparator_circuit(numbers, logic, position_register, ancilla_register, comparison_register, main_circuit):
    operations, explanations = create_combined_operations(
        numbers, QUBITS_NUM, logic, position_register, ancilla_register, comparison_register
    )
    check_all_breakpoints(main_circuit, operations)
    
    return explanations 

In [5]:
def initialize_gaussian_wavepacket(x_0, p_0, δ):
    psi = np.zeros(N, dtype=complex)
    norm_factor = (1/(2*np.pi*δ**2))**(1/4)

    for position in range(N):
        x = -d + position * Δ_x

        gaussian = np.exp(-(x - x_0)**2 / (4*δ**2))
        phase = np.exp(1j * p_0 * (x - x_0))
        
        psi[position] = gaussian * phase

    psi = norm_factor * psi
    norm = np.sqrt(np.sum(np.abs(psi)**2))  
    psi = psi / norm
    
    return psi

In [6]:
# Simulate all possible 8-bit inputs (0 to 255)
def simulate_quantum_superposition():
    backend = Aer.get_backend('qasm_simulator')
    position_register = QuantumRegister(QUBITS_NUM, name="position")  # 
    ancilla_register = QuantumRegister(ANCILLARY_QUBITS_NUM, name="ancilla")
    comparison_register = QuantumRegister(COMPARATOR_QUBITS_NUM, name="comparison") 
    
    initial_state = initialize_gaussian_wavepacket(x_0=x_0, p_0=p_0, δ=δ)
    psi = np.zeros(2**total_qubits, dtype=complex)
    psi[:2**QUBITS_NUM] = initial_state
    
    main_circuit = QuantumCircuit(position_register, ancilla_register, comparison_register)
    all_qubits = list(position_register) + list(ancilla_register) + list(comparison_register)
    main_circuit.initialize(psi, all_qubits)
    
    # Inside simulate_quantum_superposition:
    breakpoint_logic = logics(position_register, ancilla_register, comparison_register)
    explanations = comparator_circuit(numbers, breakpoint_logic, position_register, ancilla_register, comparison_register, main_circuit)

    print("LOGIC EXPLANATIONS:")
    for number, explanation in explanations.items():
        print(f"For breakpoint {number}:")
        print(explanation)
        print()
        
    simulator = Aer.get_backend('statevector_simulator')
    result = simulator.run(main_circuit).result()
    compstate = Statevector(result.get_statevector()).data
    islessthan = Statevector(compstate).probabilities([QUBITS_NUM+ANCILLARY_QUBITS_NUM])
    islessthan_1 = Statevector(compstate).probabilities([QUBITS_NUM+ANCILLARY_QUBITS_NUM+1])
    print(f"Comparison[0] probabilities for WAVEFXN - |0⟩: {islessthan[0]:.6f}, |1⟩: {islessthan[1]:.6f}")
    print(f"Comparison[1] probabilities for WAVEFXN - |0⟩: {islessthan_1[0]:.6f}, |1⟩: {islessthan_1[1]:.6f}")
    
    for x in range(2**QUBITS_NUM):  # Loop over all 8-bit numbers
        test_circuit = QuantumCircuit(position_register, ancilla_register, comparison_register)
        
        # Encode x as a quantum state using a delta function (bit-flip where needed)
        for i in range(QUBITS_NUM):
            if (x >> i) & 1:  # If bit i of x is 1, apply X-gate
                test_circuit.x(position_register[i])
        
        # Apply the quantum comparison circuit
        comparator_circuit(numbers, breakpoint_logic, position_register, ancilla_register, comparison_register, test_circuit)
        
        simulator = Aer.get_backend('statevector_simulator')
        result = simulator.run(test_circuit).result()
        compstate = Statevector(result.get_statevector()).data
        islessthan = Statevector(compstate).probabilities([QUBITS_NUM+ANCILLARY_QUBITS_NUM])
        islessthan_1 = Statevector(compstate).probabilities([QUBITS_NUM+ANCILLARY_QUBITS_NUM+1])
        print(f"Comparison[0] probabilities for input {x}- |0⟩: {islessthan[0]:.6f}, |1⟩: {islessthan[1]:.6f}")
        print(f"Comparison[1] probabilities for input {x}- |0⟩: {islessthan_1[0]:.6f}, |1⟩: {islessthan_1[1]:.6f}")

# Run the simulation
simulate_quantum_superposition()

LOGIC EXPLANATIONS:
For breakpoint 68:
Number: 68 (binary: 01000100)
------------------------------------------------------------
For a 8-bit string to be LESS THAN 68:
b_7 = 0 AND b_6 = 0 OR (b_5 = 0 AND b_4 = 0 AND b_3 = 0 AND b_2 = 0)


For breakpoint 96:
Number: 96 (binary: 01100000)
------------------------------------------------------------
For a 8-bit string to be LESS THAN 96:
b_7 = 0 AND b_6 = 0 OR (b_5 = 0)


Comparison[0] probabilities for WAVEFXN - |0⟩: 0.862672, |1⟩: 0.137328
Comparison[1] probabilities for WAVEFXN - |0⟩: 0.264881, |1⟩: 0.735119
Comparison[0] probabilities for input 0- |0⟩: 0.000000, |1⟩: 1.000000
Comparison[1] probabilities for input 0- |0⟩: 0.000000, |1⟩: 1.000000
Comparison[0] probabilities for input 1- |0⟩: 0.000000, |1⟩: 1.000000
Comparison[1] probabilities for input 1- |0⟩: 0.000000, |1⟩: 1.000000
Comparison[0] probabilities for input 2- |0⟩: 0.000000, |1⟩: 1.000000
Comparison[1] probabilities for input 2- |0⟩: 0.000000, |1⟩: 1.000000
Comparison[0] 